# Fragestellung 1.4: Wie hat sich der Anteil fossiler Heizungen in den letzten 20 Jahren in einer Gemeinde verändert? 

Um diese Fragen zu beantworten, werden Daten aus dem "Gebäude- und Wohnungsregister" (GWR) verwendet. Die Daten werden nach Gemeinde, Bauperiode und Energiequelle gruppiert. Anschliessend werden die einzelnen Objekte gezählt. Anschliessend wird die resultierende Liste als CSV-Datei exportiert.

## Notebook vorbereiten und Daten importieren

In [1]:
# Zu verwendende Bibliotheken importieren
import pandas as pd
from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen

In [5]:
# Datenquelle definieren
url = urlopen("https://public.madd.bfs.admin.ch/gl.zip")

# ZIP-File herunterladen und Dataframe erstellen
zipfile = ZipFile(BytesIO(url.read()))
df = pd.read_csv(zipfile.open('gebaeude_batiment_edificio.csv'), 
                     sep='\t')

df.head()

,EGID,GDEKT,GGDENR,GGDENAME,EGRID,LGBKR,LPARZ,LPARZSX,LTYP,GEBNR,...,GWAERDATH2,GWAERZW1,GENW1,GWAERSCEW1,GWAERDATW1,GWAERZW2,GENW2,GWAERSCEW2,GWAERDATW2,GEXPDAT
0,101173552,GL,1630,Glarus Nord,CH946922708026,1608.0,654,NaN,NaN,1608555,...,NaN,7610.0,7598.0,869.0,2002-06-26,NaN,NaN,NaN,NaN,2022-10-05
1,101173553,GL,1630,Glarus Nord,CH776978052285,1608.0,493,NaN,NaN,1608554,...,NaN,7650.0,7560.0,869.0,2002-06-26,NaN,NaN,NaN,NaN,2022-10-05
2,101173601,GL,1630,Glarus Nord,CH516984227890,1608.0,657,NaN,NaN,1608562,...,NaN,7630.0,7530.0,869.0,2003-06-30,NaN,NaN,NaN,NaN,2022-10-05
3,101173612,GL,1630,Glarus Nord,CH427001226993,1608.0,662,NaN,NaN,1608576,...,NaN,7610.0,7598.0,869.0,2008-07-17,NaN,NaN,NaN,NaN,2022-10-05
4,101174404,GL,1630,Glarus Nord,CH316942227752,1617.0,2006,NaN,NaN,16172231,...,NaN,7610.0,7598.0,859.0,2021-08-26,NaN,NaN,NaN,NaN,2022-10-05


In [3]:
# Alle Attribute und Datentypen der importierten Daten anschauen.
df.dtypes

EGID            int64
GDEKT          object
GGDENR          int64
GGDENAME       object
EGRID          object
LGBKR         float64
LPARZ          object
LPARZSX       float64
LTYP          float64
GEBNR         float64
GBEZ           object
GKODE         float64
GKODN         float64
GKSCE           int64
GSTAT           int64
GKAT            int64
GKLAS         float64
GBAUJ         float64
GBAUM         float64
GBAUP         float64
GABBJ         float64
GAREA         float64
GVOL          float64
GVOLNORM      float64
GVOLSCE       float64
GASTW         float64
GANZWHG       float64
GAZZI         float64
GSCHUTZR      float64
GEBF          float64
GWAERZH1      float64
GENH1         float64
GWAERSCEH1    float64
GWAERDATH1     object
GWAERZH2      float64
GENH2         float64
GWAERSCEH2    float64
GWAERDATH2     object
GWAERZW1      float64
GENW1         float64
GWAERSCEW1    float64
GWAERDATW1     object
GWAERZW2      float64
GENW2         float64
GWAERSCEW2    float64
GWAERDATW2

## Daten vorbereiten

In [4]:
# Die gewünschten Attribute sinnvoll umbenennen und in ein neues Dateframe speichern.
buildings = df.rename(columns={"GDEKT":"Kanton",
                               "GGDENAME": "Gemeinde",
                               "GKODE": "lat",
                               "GKODN": "lon",
                               "GKAT":"Gebaeudekategorie",
                               "GBAUJ":"Baujahr",
                               "GBAUP":"Bauperiode",
                               "GANZWHG":"AnzahlWohnungen",
                               "GEBF":"Energiebezugsflaeche",
                               "GWAERZH1":"Waermeerzeuger",
                               "GENH1":"Energiequelle",
                               "GWAERSCEH1":"InfoquelleHeizung"})

In [5]:
# Die nicht verwendeten Attribute löschen
buildings = buildings.drop(['GGDENR','EGRID','LGBKR','LPARZ','LPARZSX','LTYP','GEBNR','GBEZ','GKSCE','GSTAT','GKLAS','GBAUM','GABBJ','GAREA','GVOL','GVOLNORM','GVOLSCE','GASTW','GAZZI','GSCHUTZR','GWAERDATH1','GWAERZH2','GENH2','GWAERSCEH2','GWAERDATH2','GWAERZW1','GENW1','GWAERSCEW1','GWAERDATW1','GWAERZW2','GENW2','GWAERSCEW2','GWAERDATW2','GEXPDAT'], axis=1)

In [6]:
# Nullwerte anschauen
buildings_null = buildings.isnull().sum()
buildings_null

EGID                        0
Kanton                      0
Gemeinde                    0
lat                         0
lon                         0
Gebaeudekategorie           0
Baujahr                 26090
Bauperiode               3795
AnzahlWohnungen         23509
Energiebezugsflaeche    44608
Waermeerzeuger          21585
Energiequelle           21524
InfoquelleHeizung       21514
dtype: int64

In [7]:
# Die korrekten Bezeichnungen den Heizcodes zuweisen und als Liste speichern.
buildings['Energiequelle'] = buildings.Energiequelle.replace({
                                            7500: 'Keine',
                                            7501: 'Luft',
                                            7510: 'Erdwärme',
                                            7511: 'Erdwärme',
                                            7512: 'Erdwärme',
                                            7513: 'Wasser',
                                            7520: 'Gas',
                                            7530: 'Heizöl',
                                            7540: 'Holz',
                                            7541: 'Holz',
                                            7542: 'Holz',
                                            7543: 'Holz',
                                            7550: 'Abwärme',
                                            7560: 'Elektrizität',
                                            7570: 'Sonne',
                                            7580: 'Fernwärme',
                                            7581: 'Ferwärme',
                                            7582: 'Fernwärme',
                                            7598: 'Unbestimmt',
                                            7599: 'Keine'
                                            })


In [8]:
# Bereinigtes DataFrame anschauen
buildings

,EGID,Kanton,Gemeinde,lat,lon,Gebaeudekategorie,Baujahr,Bauperiode,AnzahlWohnungen,Energiebezugsflaeche,Waermeerzeuger,Energiequelle,InfoquelleHeizung
0,101141950,JU,Boécourt,2582709.683,1244193.026,1020,2000.0,8019.0,1.0,196.0,7410.0,Erdwärme,869.0
1,101141951,JU,Boécourt,2582783.229,1244114.144,1020,2001.0,8020.0,1.0,215.0,7410.0,Luft,869.0
2,101141952,JU,Boécourt,2583210.113,1244847.783,1020,2002.0,8020.0,1.0,NaN,7430.0,Heizöl,869.0
3,101141953,JU,Boécourt,2583144.382,1244780.399,1020,2001.0,8020.0,1.0,NaN,7410.0,Luft,869.0
4,11514675,JU,Boécourt,2582961.337,1244222.273,1020,NaN,8019.0,1.0,NaN,7410.0,Luft,860.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
47461,984473,JU,La Baroche,2582768.637,1251359.261,1020,NaN,8013.0,1.0,NaN,7430.0,Heizöl,860.0
47462,984474,JU,La Baroche,2582743.072,1251406.135,1040,NaN,8015.0,1.0,NaN,7430.0,Heizöl,860.0
47463,984475,JU,La Baroche,2584600.501,1251495.804,1020,NaN,8019.0,1.0,NaN,7436.0,Gas,860.0
47464,984476,JU,La Baroche,2583393.948,1251448.443,1020,NaN,8011.0,1.0,NaN,7436.0,Holz,860.0


## Anzahl Gebäude pro Gemeinde, Bauperiode und Energiequelle zählen und als DataFrame speichern

In [9]:
# Zählt alle Gebäude pro Gemeinde, Bauperiode und Energiequelle
Gemeindeliste = buildings[['Gemeinde','Bauperiode','Energiequelle']].value_counts().reset_index()

In [10]:
# Die Spaltennamen des DataFrames neu benennen
Gemeindeliste.columns  = ['Gemeinde','Bauperiode','Energiequelle','Anzahl']
Gemeindeliste

,Gemeinde,Bauperiode,Energiequelle,Anzahl
0,Delémont,8012.0,Gas,394
1,Haute-Sorne,8013.0,Heizöl,286
2,Delémont,8012.0,Heizöl,280
3,Porrentruy,8011.0,Fernwärme,245
4,Delémont,8013.0,Heizöl,229
...,...,...,...,...
2796,Les Breuleux,8014.0,Luft,1
2797,Courchavon,8012.0,Erdwärme,1
2798,Les Breuleux,8016.0,Fernwärme,1
2799,Courchavon,8011.0,Unbestimmt,1


## Daten als CSV-Datei exportieren

In [11]:
# Als CSV-File exportieren
Gemeindeliste.to_csv('Gemeindeliste_1-4.csv',sep=',', encoding="utf-8-sig")